In [2]:
import numpy as np

In [5]:
class NestedMonteCarlo:
    simulate_scenario : callable # function of (size)
    simulate_inner_loss : callable # function of (scenario, size)
    loss_threshold : float

    def __init__(self, simulate_scenario, simulate_inner_loss) -> None:
        self.simulate_scenario = simulate_scenario
        self.simulate_inner_loss = simulate_inner_loss

## Gaussian example

In [6]:
rng = np.random.default_rng(seed=0xBADBADBAD)

def gaussian_simulate_scenario(size, sigma_outer : float = 1, rng : np.random.Generator = rng):
    return rng.normal(loc=0, scale=sigma_outer, size=size)

def gaussian_simulate_inner_loss(size, scenario, sigma_inner : float = 5, rng : np.random.Generator = rng):
    return -scenario + rng.normal(loc=0, scale=sigma_inner, size=size)

NameError: name 'c_arr' is not defined